In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import networkx as nx
import math
import numpy as np

from sklearn.preprocessing import LabelBinarizer
from random import sample, choice
from convnet import InputLayer, GraphConvLayer, FingerprintLayer, LinearRegressionLayer
from wb2 import WeightsAndBiases
from flatten import flatten

In [27]:
def make_random_graph(nodes, n_edges, features_dict):
    """
    Makes a randomly connected graph. 
    """
    
    G = nx.Graph()
    for n in nodes:
        G.add_node(n, features=features_dict[n])
    
    for i in range(n_edges):
        u, v = sample(G.nodes(), 2)
        G.add_edge(u, v)
        
    return G

In [28]:
# features_dict will look like this:
# {0: array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
#  1: array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
#  2: array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
#  3: array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
#  4: array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
#  5: array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
#  6: array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
#  7: array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
#  8: array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
#  9: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])}

all_nodes = [i for i in range(10)]    
lb = LabelBinarizer()
features_dict = {i:lb.fit_transform(all_nodes)[i] for i in all_nodes}

G = make_random_graph(sample(all_nodes, 6), 5, features_dict)
G.edges(data=True)
# G.nodes(data=True)

[(1, 2, {}), (2, 3, {}), (3, 4, {})]

In [29]:
# def score(G):
#     """
#     The regressable score for each graph will be the sum of the 
#     (square root of each node + the sum of its neighbors.)
#     """
#     sum_score = 0
#     for n, d in G.nodes(data=True):
#         sum_score += math.sqrt(n)
        
#         for nbr in G.neighbors(n):
#             sum_score += nbr
#     return sum_score

def score(G):
    """
    This score is the number of nodes in the graph.
    """
    return (len(G.nodes()))

score(G)

6

In [30]:
n_nodes = [i for i in range(2, len(all_nodes))]
graphs = [make_random_graph(sample(all_nodes, choice(n_nodes)), choice(n_nodes), features_dict) for i in range(10)]
len(graphs)

10

In [31]:
input_shape = (1, 10)
inputs = InputLayer(input_shape).forward_pass(graphs)
layers = [GraphConvLayer(kernel_shape=(10, 20)),
          # GraphConvLayer(kernel_shape=(20, 20)),
          GraphConvLayer(kernel_shape=(20, 15)),
          FingerprintLayer(15),
          LinearRegressionLayer(shape=(15,1))]

In [32]:
def initialize_network(input_shape, layers, graphs):
    """
    Initializes all weights, biases and other parameters to random floats
    between 0 and 1.
    
    Returns a WeightsAndBiases class that stores all of the parameters
    as well.
    """
    wb_all = WeightsAndBiases()
    flatteners = dict()
    curr_shape = input_shape
    for i, layer in enumerate(layers):
        print(i, layer, curr_shape)
        curr_shape, wb = layer.build_weights(curr_shape)
        print(curr_shape)       
        wb_all['layer{0}_{1}'.format(i, layer)] = wb
    
    return wb_all

wb_all = initialize_network(input_shape, layers, graphs)
# wb_all

0 GraphConvLayer (1, 10)
(1, 20)
1 GraphConvLayer (1, 20)
(1, 15)
2 FingerprintLayer (1, 15)
(1, 15)
3 LinearRegressionLayer (1, 15)
(1, 1)


In [33]:
def predict(wb_vect, wb_unflattener, inputs, layers, graphs):
    curr_inputs = inputs
    wb_all = wb_unflattener(wb_vect)
    for i, layer in enumerate(layers):
        wb = wb_all['layer{0}_{1}'.format(i, layer)]
        curr_inputs = layer.forward_pass(wb, curr_inputs, graphs)
    return curr_inputs

predict(*wb_all.flattened(), inputs, layers, graphs)

(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
Input shape: (10, 15)
Wb shape: (15, 1)


array([[ 0.06517928],
       [ 0.03995284],
       [ 0.05630851],
       [ 0.02262357],
       [-0.05838902],
       [ 0.04559103],
       [-0.01305972],
       [-0.01239189],
       [-0.04543777],
       [-0.01293455]])

In [22]:
def train_loss(wb_vect, wb_unflattener, inputs, layers, graphs):
    """
    Training loss is MSE. 
    """
    preds = predict(wb_vect, wb_unflattener, inputs, layers, graphs)
    actual = np.array([score(g) for g in graphs])
    
    mse = np.sum(np.power(preds - actual, 2)) / len(graphs)
    
    return mse

train_loss(*wb_all.flattened(), inputs, layers, graphs)

(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
Input shape: (10, 15)
Wb shape: (15, 1)


ValueError: operands could not be broadcast together with shapes (10,1) (15,1) 

In [23]:
from autograd import grad
gradfun = grad(train_loss)
gradfun(*wb_all.flattened(), inputs, layers, graphs)

(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
Input shape: (10, 15)
Wb shape: (15, 1)


ValueError: operands could not be broadcast together with shapes (10,1) (15,1) 

In [19]:
# flattened, unflattener = wb_all.flattened()
# len(flatten(unflattener(flattened))[0])
wb_vect, wb_unflattener = wb_all.flattened()

In [20]:
def sgd(grad, wb_vect, wb_unflattener, inputs, layers, graphs, callback=None, num_iters=200, step_size=0.1, mass=0.9):
    """
    Stochastic gradient descent with momentum.
    """
    # wb_vect, wb_unflattener = wb.flattened()
    velocity = np.zeros(len(wb_vect))
    for i in range(num_iters):
        print(i)
        g = grad(wb_vect, wb_unflattener, inputs, layers, graphs)

        velocity = mass * velocity - (1.0 - mass) * g
        wb_vect += step_size * velocity
        
        print(train_loss(wb_vect, wb_unflattener, inputs, layers, graphs))
        
    # return wb_vect, wb_unflattener

sgd(gradfun, wb_vect, wb_unflattener, inputs, layers, graphs, num_iters=500, step_size=0.01)

0
227.510200937
1
198.343845475
2
144.717639159
3
100.173988145
4
185.229861088
5
80.6573383231
6
97.5794864896
7
99.3467633915
8
83.2357795659
9
63.8988303794
10
57.5255857262
11
69.7763201733
12
76.5241644995
13
66.7764784219
14
56.6204003799
15
53.1477100538
16
54.8599654403
17
57.8782616002
18
59.4010012365
19
58.4707224887
20
55.6923640508
21
52.6384304986
22
50.9985476592
23
51.3841441142
24
52.752630845
25
53.142494548
26
51.6636729701
27
49.4389510879
28
48.0181206023
29
47.7890612976
30
48.0659493067
31
47.9416668126
32
47.0198659204
33
45.6266254353
34
44.7071360773
35
44.7931064368
36
44.8164944838
37
43.8515704423
38
42.4846808784
39
41.6584890782
40
41.4533885553
41
41.1794353712
42
40.5368019413
43
40.0466614087
44
40.0424966507
45
39.9272804184
46
39.337044563
47
38.8525912372
48
38.7261259377
49
38.5072336053
50
38.0020670462
51
37.6041274484
52
37.4594468338
53
37.18547415
54
36.7344279856
55
36.4513449542
56
36.306875218
57
36.0480213548
58
35.7323405696
59
35.5588891

In [21]:
scores = [score(g) for g in graphs]
scores

[8, 5, 4, 4, 4, 3, 7, 2, 6, 5]

In [23]:
preds = predict(wb_vect, wb_unflattener, inputs, layers, graphs)
preds

array([[ 4.8],
       [ 4.8],
       [ 4.8],
       [ 4.8],
       [ 4.8],
       [ 4.8],
       [ 4.8],
       [ 4.8],
       [ 4.8],
       [ 4.8]])

In [22]:
new_graphs = [make_random_graph(sample(all_nodes, choice(n_nodes)), choice(n_nodes), features_dict) for i in range(10)]

KeyError: 'idx'